In [4]:
import tifffile
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [5]:
#Reading the input image and the PLEASANT Enhanced Image
input_im = tifffile.imread('/home/training/Harmionee/MCC_Images/MCC_MRC_20141110T172436240_D_GDS.IMG (RGB).tif')
enhanced_im  = tifffile.imread('/home/training/Harmionee/MCC_Images_8Bit/Final_Intensity_AH_GC.tif')

In [6]:
#The image i shifted by this matrix for feature matching 
#This matrix shifts the image 10 pixels to the right and 25 pixels downwards 
#negative values will shift the image towards the left and upwards
M = np.float32([
    [1,0,10],
    [0,1,25]
])

#cv2.warpAffine for shifting the image
shifted_out_im = cv2.warpAffine(enhanced_im,M,(enhanced_im.shape[1], enhanced_im.shape[0]))
shifted_in_im  = cv2.warpAffine(input_im,M,(input_im.shape[1], input_im.shape[0]))

In [7]:
#creating orb object
orb = cv2.ORB_create()

#converting the input and the output enhanced images to grayscale for feature detection
gray_in = cv2.cvtColor(input_im, cv2.COLOR_RGB2GRAY)
gray_out = cv2.cvtColor(enhanced_im, cv2.COLOR_RGB2GRAY)
gray_shifted_in = cv2.cvtColor(shifted_in_im, cv2.COLOR_RGB2GRAY)
gray_shifted_out = cv2.cvtColor(shifted_out_im, cv2.COLOR_RGB2GRAY)

#computing keypoints and descriptors
kp1, ds1 = orb.detectAndCompute(gray_in, None)
kp2, ds2 = orb.detectAndCompute(gray_shifted_in, None)
kp3, ds3 = orb.detectAndCompute(gray_out, None)
kp4, ds4 = orb.detectAndCompute(gray_shifted_out, None)

In [8]:
#total number of keypoints in the input image vs the enhanced_image
#len(kp1),len(kp3)

In [9]:
#for simply drawing keypoints on the images
#im1 = cv2.drawKeypoints(input_im, kp1, None, flags=0, color = (0,255,0))
#im2 = cv2.drawKeypoints(enhanced_im, kp2, None, flags=0, color = (0,255,0))

#cv2 matcher object for feature matching
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

#matching the keypoints using descriptors
matches_in = matcher.match(ds1,ds2)
matches_in =  sorted(matches_in, key = lambda x:x.distance)
matches_out = matcher.match(ds3,ds4)
matches_out =  sorted(matches_out, key = lambda x:x.distance)

In [10]:
#drawing the matches on the images
#can use flag attribute for representing the match points with a diff shape eg.flags = cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG 
matched_img_in = cv2.drawMatches(input_im, kp1, shifted_in_im, kp2,matches_in,None, matchColor = (0,255,0))
matched_img_out = cv2.drawMatches(enhanced_im, kp3, shifted_out_im, kp4,matches_out,None, matchColor = (0,255,0))